In [8]:
import pandas as pd
import os
import wikipedia

articles = pd.read_csv('test.csv')
articles.head()

,Rank,Page title,Views,Daily average,Assessment,Importance
0,1,"Charles, Prince of Wales",2368520,76403,B,Low
1,2,COVID-19 pandemic,1373905,44319,C,Top
2,3,COVID-19 pandemic by country and territory,1145334,36946,List,Top
3,4,Coronavirus disease 2019,815584,26309,B,Top
4,5,UFC 256,792502,25564,Start,Low


In [9]:
text = wikipedia.page("Coronavirus").content
text

'Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans and birds, they cause respiratory tract infections that can range from mild to lethal. Mild illnesses in humans include some cases of the common cold (which is also caused by other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS, and COVID-19. In cows and pigs they cause diarrhea, while in mice they cause hepatitis and encephalomyelitis.\nCoronaviruses constitute the subfamily Orthocoronavirinae, in the family Coronaviridae, order Nidovirales, and realm Riboviria. They are enveloped viruses with a positive-sense single-stranded RNA genome and a nucleocapsid of helical symmetry. The genome size of coronaviruses ranges from approximately 26 to 32 kilobases, one of the largest among RNA viruses. They have characteristic club-shaped spikes that project from their surface, which in electron micrographs create an image reminiscent of the solar corona, f

In [38]:
import re

new_text = re.sub('()[,\.!?]','', text).lower()
new_text

d = {'col1': [0], 'paper_text': [new_text]}

papers = pd.DataFrame(data = d)
papers

,col1,paper_text
0,0,coronaviruses are a group of related rna virus...


In [25]:
from wordcloud import WordCloud

In [26]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michaellam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data = papers.paper_text.tolist()
data_words = list(sent_to_words(data))

data_words = remove_stopwords(data_words)

data_words[:1][0][:30]

['coronaviruses',
 'group',
 'related',
 'rna',
 'viruses',
 'cause',
 'diseases',
 'mammals',
 'birds',
 'humans',
 'birds',
 'cause',
 'respiratory',
 'tract',
 'infections',
 'range',
 'mild',
 'lethal',
 'mild',
 'illnesses',
 'humans',
 'include',
 'cases',
 'common',
 'cold',
 'also',
 'caused',
 'viruses',
 'predominantly',
 'rhinoviruses']

In [55]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(data_words)

texts = data_words

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1][0][:30])

[(0, 4), (1, 2), (2, 1), (3, 1), (4, 3), (5, 2), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 9), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 4), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1)]


In [56]:
from pprint import pprint

num_topics = 10

lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word = id2word, num_topics = num_topics)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"coronavirus" + 0.009*"human" + 0.007*"virus" + 0.006*"coronaviruses" '
  '+ 0.005*"protein" + 0.005*"cov" + 0.005*"host" + 0.005*"proteins" + '
  '0.005*"species" + 0.004*"viruses"'),
 (1,
  '0.040*"coronavirus" + 0.015*"virus" + 0.012*"species" + '
  '0.010*"coronaviruses" + 0.010*"human" + 0.008*"rna" + 0.008*"proteins" + '
  '0.008*"protein" + 0.007*"cell" + 0.007*"cov"'),
 (2,
  '0.025*"coronavirus" + 0.012*"coronaviruses" + 0.011*"virus" + '
  '0.008*"species" + 0.008*"cov" + 0.008*"host" + 0.008*"proteins" + '
  '0.007*"human" + 0.007*"viruses" + 0.006*"sars"'),
 (3,
  '0.022*"coronavirus" + 0.013*"coronaviruses" + 0.012*"human" + 0.011*"virus" '
  '+ 0.009*"cov" + 0.008*"species" + 0.008*"rna" + 0.007*"protein" + '
  '0.007*"host" + 0.006*"proteins"'),
 (4,
  '0.028*"coronavirus" + 0.017*"virus" + 0.013*"coronaviruses" + '
  '0.011*"species" + 0.010*"human" + 0.009*"protein" + 0.008*"proteins" + '
  '0.007*"rna" + 0.007*"cov" + 0.007*"host"'),
 (5,
  '0.023*"coron

In [57]:
import pyLDAvis.gensim
import pickle
import pyLDAvis

In [58]:
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

if True:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
    
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_' + str(num_topics) + '.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.010202  0.001623       1        1  99.968035
1      0.005946 -0.002549       2        1   0.003552
4      0.002747 -0.000993       3        1   0.003552
3      0.000647  0.001366       4        1   0.003552
8      0.000290 -0.000593       5        1   0.003552
9     -0.000167  0.001676       6        1   0.003552
5     -0.000559  0.000571       7        1   0.003552
2     -0.001048 -0.000701       8        1   0.003552
6     -0.005679 -0.000664       9        1   0.003551
0     -0.012378  0.000265      10        1   0.003551, topic_info=                 Term       Freq      Total Category  logprob  loglift
210       coronavirus  97.000000  97.000000  Default  30.0000  30.0000
1038            virus  44.000000  44.000000  Default  29.0000  29.0000
457             human  38.000000  38.000000  Default  28.0000  28.0000
211     coronaviruses  35.000000  35.000000  Default  27.0000  27.0000
216               cov  37.000000  37.000000  Default  26.0000  26.0000
817               rna  27.000000  27.000000  Default  25.0000  25.0000
751           protein  27.000000  27.000000  Default  24.0000  24.0000
883           species  25.000000  25.000000  Default  23.0000  23.0000
454              host  25.000000  25.000000  Default  22.0000  22.0000
828              sars  20.000000  20.000000  Default  21.0000  21.0000
1039          viruses  20.000000  20.000000  Default  20.0000  20.0000
752          proteins  19.000000  19.000000  Default  19.0000  19.0000
805       respiratory  17.000000  17.000000  Default  18.0000  18.0000
414            genome  17.000000  17.000000  Default  17.0000  17.0000
79                bat  15.000000  15.000000  Default  16.0000  16.0000
184            common  14.000000  14.000000  Default  15.0000  15.0000
144              cell  13.000000  13.000000  Default  14.0000  14.0000
785           related  13.000000  13.000000  Default  13.0000  13.0000
446               hku  13.000000  13.000000  Default  12.0000  12.0000
1033            viral  12.000000  12.000000  Default  11.0000  11.0000
139            causes  12.000000  12.000000  Default  10.0000  10.0000
37               also  12.000000  12.000000  Default   9.0000   9.0000
656                oc  11.000000  11.000000  Default   8.0000   8.0000
177              cold  11.000000  11.000000  Default   7.0000   7.0000
431              hcov  11.000000  11.000000  Default   6.0000   6.0000
482            infect  11.000000  11.000000  Default   5.0000   5.0000
551              like  11.000000  11.000000  Default   4.0000   4.0000
1000              two  10.000000  10.000000  Default   3.0000   3.0000
261         different  11.000000  11.000000  Default   2.0000   2.0000
326          envelope  10.000000  10.000000  Default   1.0000   1.0000
...               ...        ...        ...      ...      ...      ...
454              host   0.000504  25.406639  Topic10  -5.2911  -0.5826
785           related   0.000336  13.235310  Topic10  -5.6950  -0.3345
654            number   0.000241   7.699275  Topic10  -6.0278  -0.1255
769           reading   0.000231   7.206121  Topic10  -6.0706  -0.1021
80               bats   0.000219   6.625618  Topic10  -6.1247  -0.0723
144              cell   0.000333  13.395646  Topic10  -5.7049  -0.3564
211     coronaviruses   0.000600  35.869403  Topic10  -5.1171  -0.7535
433            health   0.000185   5.029460  Topic10  -6.2933   0.0348
883           species   0.000481  25.444759  Topic10  -5.3377  -0.6308
261         different   0.000293  11.076930  Topic10  -5.8317  -0.2931
952          terminal   0.000237   7.750261  Topic10  -6.0442  -0.1485
1038            virus   0.000654  44.760989  Topic10  -5.0308  -0.8887
390             forms   0.000165   4.158179  Topic10  -6.4102   0.1081
1039          viruses   0.000412  20.680913  Topic10  -5.4924  -0.5782
446               hku   0.000312  13.152677